# Fine-tuning Passage Embeddings with GenQ

State-of-the-art semantic search applications like [extractive question answering](https://docs.pinecone.io/docs/extractive-question-answering) and [Retrieval Augmented Generation](https://arxiv.org/abs/2005.11401) (RAG) have gained significant attention recently, allowing users to query a large document corpus using natural language questions. One of the most important components in these systems is the **embedding model**, which maps input text into a vector, providing a numeric representation of the semantic meaning of the text. An effective model is tuned to place passages with similar meaning in similar locations in the embedding space. In a process known as **retrieval**, this space is efficiently interrogated to find the most relevant documents given an input query.

In practice, general-purpose language models do not perform well on this retrieval task, particularly when dealing with documents related to a specialized domain, like medicine or law. The typical solution is to "fine-tune" a generic model to the particular task and field of interest by feeding it thousands of labeled training examples. For a semantic search engine, the most valuable dataset would be human-generated (query, passage) pairs.

For example, this excerpt from the [Sagemaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/train-model.html) contains an appropriate answer for the following query:

```
Query: What instance type should I use for an NLP model?

Relevant Passage: Choose the right instance type and infrastructure management tools for your use case. You can start from a small instance and scale up depending on your workload. For training a model on a tabular dataset, start with the smallest CPU instance of the C4 or C5 instance families. For training a large model for computer vision or natural language processing, start with the smallest GPU instance of the P2, P3, G4dn or G5 instance families.
```

However, it would be extremely expensive and time consuming to have humans generate questions for thousands of documents. In this series of notebooks, we will demonstrate an approach called [**GenQ**](https://www.sbert.net/examples/unsupervised_learning/query_generation/README.html), in which we leverage a separate model to automatically generate so-called "synthetic" queries for our dataset. For more details about GenQ, see the [NIPS publication](https://arxiv.org/abs/2104.08663).

This demonstration is broken into 3 notebooks which are designed to be completed in order.

1. **01-Data-Preparation** (this notebook): initialization, dataset preparation, generating synthetic queries
2. 02-Finetune-and-Deploy-Model: finetuning the embedding model and deploying it to a Sagemaker endpoint
3. 03-Applications-and-Evaluation: comparing the finetuned model to a baseline and using it for RAG and extractive QA

Throughout the demo, we will leverage frameworks like [LangChain](https://python.langchain.com/en/latest/) and [Hugging Face](https://huggingface.co/) to simplify and operationalize the workflow. We will also use [Amazon SageMaker](https://aws.amazon.com/sagemaker/) for fully-managed and scalable inference, training, and deployment.

## Part 1: Data Preparation

In this notebook, we will download a document dataset, break it into individual passages, and use a GenQ model to generate synthetic queries for each excerpt.

## 1.0 Install dependencies

**Make sure to use the `conda_pytorch_p39` SageMaker kernel, which comes with many dependencies pre-installed.** We have added a few additional packages to the `requirements.txt` file.

In [ ]:
!pip install -r requirements.txt -q

## 1.1. Load and preprocess dataset

We will use pubmed abstracts as an example of a domain-specific corpus that may not produce great retrieval results with generic embedding models.

In [3]:
from datasets import load_dataset
from IPython.core.display import display, HTML
import os

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# load abstracts of pubmed articles
dataset = load_dataset('ccdv/pubmed-summarization', 'document', split='train[:2500]')
articles = [{'id': str(i).zfill(6), 'abstract': d['abstract']} for i, d in enumerate(dataset)]

print(f"Number of articles: {len(articles)}")

# print an example article
display(HTML(articles[0]['abstract']))

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset pubmed-summarization downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/ccdv___pubmed-summarization/document/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b. Subsequent calls will reuse this data.
Number of articles: 2500


In semantic search applications, it is common practice to split the input documents into smaller passages. The size of the excerpts requires you to strike a balance between accuracy and specificity - for example, larger passages are more likely to contain a full answer to your query, but they take longer for users (and downstream models) to parse. The most appropriate `chunk_size` will depend on your dataset and processing pipeline. LangChain supports a number of different [Text Splitters](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) - in this case we will use the [`NLTKTextSplitter`](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/nltk.html), which uses a tokenizer to split input texts.

In [4]:
# preprocessing - create passages by splitting articles
from langchain.text_splitter import NLTKTextSplitter
import nltk

nltk.download('punkt')
separator = '\n\n'
text_splitter = NLTKTextSplitter(separator=separator, chunk_size=1000)

passages, metadatas = [], []

for a in articles:
    split_texts = text_splitter.split_text(a['abstract'])
    for p in split_texts:
        new_passages = p.split(separator)
        passages += new_passages
        metadatas += [{'source': a['id']} for _ in new_passages]

print(f'Number of passages: {len(passages)}\n\n')

print('Example Passages:\n')
for p in passages[:10]:
    print(p + '\n')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Created a chunk of size 1199, which is longer than the specified 1000
Created a chunk of size 1050, which is longer than the specified 1000
Created a chunk of size 1346, which is longer than the specified 1000
Created a chunk of size 1313, which is longer than the specified 1000
Created a chunk of size 1160, which is longer than the specified 1000


Number of passages: 20454


Example Passages:

background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz , iran .

the project provided nutritious snacks in public schools over a 2-year period along with advocacy oriented actions in order to implement and promote nutritional intervention .

for evaluation of effectiveness of the intervention growth monitoring indices of pre- and post - intervention were statistically compared.results:the frequency of subjects with body mass index lower than 5% decreased significantly after intervention among girls ( p = 0.02 ) .

however , there were no significant changes among boys or total populati

## 1.2. Prepare GenQ model and use it to generate potential queries for each passage

We will use [`BeIR/query-gen-msmarco-t5-large-v1`](https://huggingface.co/BeIR/query-gen-msmarco-t5-large-v1), which is a T5-based model used for synthetic query generation tasks. The Sagemaker [Hugging Face](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html) integration will greatly simplify this deployment. To define the [`HuggingFaceModel`](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model), we need to supply an appropriate image URI, environment variables for the model and task, and an appropriate execution role.

In [5]:
# prepare (query, passage) pairs with GenQ
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()
hub = {
    'HF_MODEL_ID':'BeIR/query-gen-msmarco-t5-large-v1',
    'HF_TASK':'text2text-generation'
}

transformers_version = '4.26.0'
pytorch_version = '1.13.1'
py_version = 'py39'
use_gpu = True

# https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-inference-containers
image_uri = f"763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:{pytorch_version}-transformers{transformers_version}-{'gpu' if use_gpu else 'cpu'}-{py_version}{'-cu117' if use_gpu else ''}-ubuntu20.04"

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=image_uri,
    env=hub,
    role=role
)

Now that the GenQ model has been defined, we can use it to generate (query, passage) pairs for each passage. Because we only need the model to be temporarily deployed for this specific task, it is an excellent candidate for a [SageMaker Transform Job](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html). After generating text for our input dataset, the resources will automatically be shut down to limit costs.

**Setup SageMaker Batch Transform Job to Generate Queries Asynchronously**

At this time, the [Hugging Face Inference DLC only supports the `.jsonl` format](https://huggingface.co/docs/sagemaker/inference#run-batch-transform-with-transformers-and-sagemaker) for batch transforms. We will pass in additional parameters to ensure that 3 concise queries are generated for each passage. The `input.jsonl` file will be copied to S3.

In [6]:
import json

with open('input.jsonl', 'w') as f:
    for p in passages:
        f.write(json.dumps({
            'inputs': p,
            'parameters': {
                'max_length': 32,
                'num_return_sequences': 3,
                'top_p': 0.95,
                'do_sample': True
            }}) + '\n')

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'pubmed-finetuning'

In [7]:
!aws s3 cp input.jsonl s3://{bucket}/{prefix}/data/batch_input/input.jsonl

upload: ./input.jsonl to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/batch_input/input.jsonl


See the [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) for more details about the configuration of the transform job. In this case, we use a `join_source` on the input to ensure that we keep the (query, passage) pairs together.

In [ ]:
# transform job takes about 3 hours with p3.2xlarge

batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    strategy='SingleRecord',
    accept='application/json',
    assemble_with='Line',
    output_path=f's3://{bucket}/{prefix}/data/batch_output'
)

# starts batch transform job and uses S3 data as input
batch_job.transform(
    data=f's3://{bucket}/{prefix}/data/batch_input/input.jsonl',
    content_type='application/json',
    split_type='Line',
    join_source='Input'
)

**(Optional) Deploy query generation model to SageMaker endpoint**

As an alternative to the batch transform job, you can also deploy the model to a SageMaker endpoint using the SDK. This is useful for rapid testing - see an example of the generated queries for this passage.

In [10]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.g4dn.xlarge', # ec2 instance type
)

# synchronous inference
psg = passages[0]

queries = predictor.predict({
    'inputs': psg,
    'parameters': {
        'max_length': 32,
        'num_return_sequences': 3,
        'top_p': 0.95,
        'do_sample': True
    }
})

print(f'\nPassage:\n{psg}\n\nGenerated Queries:\n')
for q in queries:
    print(q['generated_text'])

# make sure to delete the endpoint when you're done to limit cost
# predictor.delete_endpoint()


Passage:
background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz , iran .

Generated Queries:

what is nutritional intervention based on advocacy
why community nutrition approach
what is nutrition advocacy intervention


## 1.4. Perform some light transforms on the output to turn into individual (query, passage) pairs

Once the queries have been generated, we need to do some light transformations to parse the batch output into a format suitable for the finetuning job. The training pairs will be batched into `.tsv` files containing 1024 samples each. In the next notebook, we will use this dataset to finetune a generic embedding model.

In [12]:
# transform outputs into format suitable for training job
local_path = 'batch_output'

sagemaker.s3.S3Downloader.download(f'{batch_job.output_path}/input.jsonl.out', local_path)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [13]:
from tqdm.auto import tqdm

training_data_path = 'pubmed_training_pairs'

if os.path.isdir(training_data_path) is False:
    os.mkdir(training_data_path)

pairs = []
file_count = 0

with open(f'{local_path}/input.jsonl.out', 'r') as f:
    for r in tqdm(f.readlines()):
        obj = json.loads(r)
        psg = obj['inputs'].replace('\t', ' ').replace('\n', ' ')

        for q in obj['SageMakerOutput']:
            pairs.append(q['generated_text'].replace('\t', ' ') + '\t' + psg)

        if len(pairs) > 1024:
            with open(f'{training_data_path}/pairs_{file_count:04}.tsv', 'w', encoding='utf-8') as fp:
                fp.write('\n'.join(pairs))

            file_count += 1
            pairs = []

if pairs is not None:
    # save the final, smaller than 1024 batch
    with open(f'{training_data_path}/pairs_{file_count:04}.tsv', 'w', encoding='utf-8') as fp:
        fp.write('\n'.join(pairs))

  0%|          | 0/18773 [00:00<?, ?it/s]

In [14]:
!aws s3 cp pubmed_training_pairs s3://{bucket}/{prefix}/data/training --recursive

upload: pubmed_training_pairs/pairs_0004.tsv to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/training/pairs_0004.tsv
upload: pubmed_training_pairs/pairs_0006.tsv to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/training/pairs_0006.tsv
upload: pubmed_training_pairs/pairs_0009.tsv to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/training/pairs_0009.tsv
upload: pubmed_training_pairs/pairs_0005.tsv to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/training/pairs_0005.tsv
upload: pubmed_training_pairs/pairs_0001.tsv to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/training/pairs_0001.tsv
upload: pubmed_training_pairs/pairs_0008.tsv to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/training/pairs_0008.tsv
upload: pubmed_training_pairs/pairs_0007.tsv to s3://sagemaker-us-east-1-352937523354/pubmed-finetuning/data/training/pairs_0007.tsv
upload: pubmed_training_pairs/pairs_0012.tsv to s3://sagemaker-us-eas